In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [5]:
import numpy as np
from kalman_estimation import Kalman4FROLS, Selector, get_mat_data

In [106]:
# !非线性模型
# *非线性数据
terms_path = '../data/linear_terms10D_0.50trial24.mat'
term = Selector(terms_path)
_ = term.make_terms()

# # *保存候选项集合
# # fname = './data/nonlinear_candidate_terms.txt'
# fname = './data/longlag_nonlinear_candidate_terms.txt'
# np.savetxt(fname, terms_repr, fmt='%s')

# *selection
normalized_signals, Kalman_H, candidate_terms, Kalman_S_No = term.make_selection()

# *构造 Kalman Filter
kf = Kalman4FROLS(normalized_signals, Kalman_H=Kalman_H, uc=0.01)
y_coef = kf.estimate_coef()
print(y_coef)

[[ 1.31689746 -0.88249692 -0.00705094  0.01153373  0.01596134]
 [ 0.85920768  0.03446802  0.04001234 -0.01373683  0.02436873]
 [ 0.02562259  0.84181459  0.03823484 -0.04389787  0.03792626]
 [-0.84437384  0.0120928  -0.03305907 -0.03405536  0.01621327]
 [-0.02753297  0.80194159  0.02365222 -0.23736773 -0.0259009 ]
 [-0.81876557 -0.02440997 -0.03343491 -0.0126235  -0.04247696]
 [ 0.95098928 -1.00494034  0.07815436 -0.05348226 -0.05207683]
 [-0.03093557 -0.73723245  0.38822377  0.23562854  0.02283931]
 [ 0.03932088 -0.36590503  0.36769913  0.04399609 -0.05922883]
 [-0.02863592 -0.66863637  0.0427592   0.02385565 -0.0433274 ]]


- 形式参数
    - noise_var
    - trial
    - ndim
    - type
    - uc

In [167]:
def corr_term(y_coef, terms_set, Kalman_S_No, var_name: str = 'x', step_name: str = 't'):
    n_dim, n_term = y_coef.shape
    Kalman_S_No = np.sort(Kalman_S_No)
    func_repr = []
    for var in range(n_dim):
        y = {}
        for term in range(n_term):
            y[terms_set[Kalman_S_No[var, term]]] = y_coef[var, term]
        func_repr.append(y)
    return func_repr

In [168]:
def frokf(noise_var, trial, ndim, dtype, terms, length, root='../data/', uc=0.01):
    assert dtype in ['linear', 'nonlinear'], 'type not support!'
    terms_path = root + f'{dtype}_terms{ndim}D_{noise_var:2.2f}trial{trial}.mat'
    term = Selector(terms_path)
    _ = term.make_terms()
    normalized_signals, Kalman_H, candidate_terms, Kalman_S_No = term.make_selection()
    kf = Kalman4FROLS(normalized_signals, Kalman_H=Kalman_H, uc=uc)
    y_coef = kf.estimate_coef()
    terms_set = corr_term(y_coef, candidate_terms, Kalman_S_No)
    flatten_coef, t = [], 0
    for i in range(ndim):
        flatten_coef.extend([terms_set[i][k] for k in terms[t:t+length[i]]])
        t += length[i]
    return flatten_coef

In [169]:
con_terms_linear10 = ['x1(t-1)', 'x1(t-2)', 'x1(t-2)', 'x2(t-3)', 'x1(t-2)', 'x4(t-4)', 'x9(t-2)', 'x4(t-4)', 'x1(t-1)', 'x1(t-2)', 'x7(t-2)', 
                      'x8(t-3)', 'x9(t-3)', 'x8(t-3)', 'x9(t-3)', 'x7(t-4)']  # 16
con_terms10 = [2, 1, 1, 1, 2, 1, 2, 3, 2, 1]

In [170]:
frokf(0.5, 24, 10, 'linear', con_terms_linear10, con_terms10)

[1.3141547845426411,
 -0.8804174078314431,
 0.858740029527661,
 0.8424137516067541,
 -0.8444020811678504,
 0.8023847374043557,
 -0.23757259452488289,
 -0.8189505388500772,
 0.9533506486274645,
 -1.008815562381101,
 -0.7350306647401716,
 0.3865331184451789,
 0.23587828043024994,
 -0.36567205671621256,
 0.36738771958190397,
 -0.6688729903288174]